In [41]:

import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
from datetime import date
import numpy as np
# import seaborn as sns
import matplotlib.ticker as tick
import requests
import difflib as diff
import re
import csv
import ast
import math
import os
import random
# Check the length of the code after tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

In [42]:
# language = "Java"
language = "TypeScript"
b_remove_comments = True
b_shrink_indentation = True

In [47]:
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
print(quicksort(arr))

[1, 1, 2, 3, 6, 8, 10]


In [43]:
short_name = {"JavaScript": "JS", "Python": "PY", "TypeScript": "TS", "CPP": "CPP", "C": "C", "Java": "JAVA", "C#": "CS"}
sname = short_name[language]
path_dict = {
    "JavaScript": "/hpcfs/users/a1232991/Data/CVESingle/CVE_JS/JavaScript_data.parquet",
    "Python": "/hpcfs/users/a1232991/Data/CVESingle/CVE_PY/Python_data.parquet",
    "TypeScript": "/hpcfs/users/a1232991/Data/CVESingle/CVE_TS/TypeScript_data.parquet",
    "CPP": "/hpcfs/users/a1232991/Data/CVESingle/CVE_CPP/C++_data.parquet",
    "C": "/hpcfs/users/a1232991/Data/CVESingle/CVE_C/C_data.parquet",
    "Java": "/hpcfs/users/a1232991/Data/CVESingle/CVE_JAVA/Java_data.parquet",
    "C#": "/hpcfs/users/a1232991/Data/CVESingle/CVE_CS/C#_data.parquet"
}

df_codes = pd.read_parquet(path_dict[language], engine='pyarrow')
# df_codes = df_codes[df_codes['before_change']=='True']
# df_codes = df_codes[df_codes['added_only']=='True']
df_codes = df_codes[["repo_url", "hash", "code", "label", 'added_only', 'mod_lines']]
df_codes.columns = ['repo_url', 'hash', 'code_before', 'target', 'added_only', 'mod_lines']
df_codes['commit_url'] = df_codes.apply(lambda x: x['repo_url'].replace(".git", "") + '/commit/'+x['hash'], axis=1)
# convert label to int
df_codes['target'] = df_codes['target'].astype(int)

In [44]:
idx = 265
print(df_codes[df_codes['target']==1].iloc[idx]['code_before'])
print(df_codes[df_codes['target']==1].iloc[idx]['mod_lines'])
print(df_codes[df_codes['target']==1].iloc[idx]['added_only'])
print(df_codes[df_codes['target']==1].iloc[idx]['commit_url'])

IndexError: single positional indexer is out-of-bounds

In [ ]:
# df_test = df_codes[df_codes['target']=="1"].iloc[idx:idx+1]

In [ ]:
# df_test

In [ ]:
# FLAW_TAG_SYMBOL = '\u25C6'
# POSITION = 'BEGIN'
# idx = 0

# df_test = df_test.apply(insert_tag, axis=1)
# df_test['code_before'] = df_test['code_before'].apply(lambda x: parser.clean(x))
# df_test = df_test.apply(remove_tag_and_generate_mod_lines, axis=1)

# print(df_test[df_test['target']=="1"].iloc[idx]['code_before'])
# print(df_test[df_test['target']=="1"].iloc[idx]['mod_lines'])
# print(df_test[df_test['target']=="1"].iloc[idx]['added_only'])
# print(df_test[df_test['target']=="1"].iloc[idx]['commit_url'])

In [ ]:
# FLAW_TAG_SYMBOL = '## FLAW_LINE_TAG ##'
# POSITION = 'END'
    
# # df_test = df_test.apply(insert_tag, axis=1)
# # df_test['code_before'] = df_test['code_before'].apply(auto_shrink_indentation)
# df_test = df_test.apply(remove_tag_and_generate_mod_lines, axis=1)

# print(df_test[df_test['target']=="1"].iloc[idx]['code_before'])
# print(df_test[df_test['target']=="1"].iloc[idx]['mod_lines'])
# print(df_test[df_test['target']=="1"].iloc[idx]['added_only'])
# print(df_test[df_test['target']=="1"].iloc[idx]['commit_url'])

In [ ]:
df_codes[df_codes['mod_lines'] != ''].head(10)

,repo_url,hash,code_before,target,added_only,mod_lines,commit_url
0,https://github.com/uzbl/uzbl.git,1958b52d41cba96956dc1995660de49525ed1047,"eval_js(WebKitWebView * web_view, gchar *scrip...",1,False,"4,17,18,19,20,21,22,40,41,42",https://github.com/uzbl/uzbl/commit/1958b52d41...
134,https://github.com/memcached/memcached.git,75cc83685e103bc8ba380a57468c8f04413033f9,static int try_read_command(conn *c) {\n as...,1,False,89,https://github.com/memcached/memcached/commit/...
135,https://github.com/memcached/memcached.git,75cc83685e103bc8ba380a57468c8f04413033f9,static enum try_read_result try_read_network(c...,1,False,3,https://github.com/memcached/memcached/commit/...
299,https://github.com/memcached/memcached.git,d9cd01ede97f4145af9781d448c62a3318952719,static int try_read_command(conn *c) {\n as...,1,False,101,https://github.com/memcached/memcached/commit/...
388,https://github.com/memcached/memcached.git,a8c4a82787b8b6c256d61bd5c42fb7f92d1bae00,static inline void process_get_command(conn *c...,1,False,23,https://github.com/memcached/memcached/commit/...
497,https://github.com/memcached/memcached.git,dbb7a8af90054bf4ef51f5814ef7ceb17d83d974,static void settings_init(void) {\n setting...,1,False,4,https://github.com/memcached/memcached/commit/...
655,https://github.com/memcached/memcached.git,554b56687a19300a75ec24184746b5512580c819,static inline void get_conn_text(const conn *c...,1,False,"33,34",https://github.com/memcached/memcached/commit/...
847,https://github.com/uclouvain/openjpeg.git,5d00b719f4b93b1445e6fb4c766b9a9883c57949,void opj_get_all_encoding_parameters( const ...,1,False,"42,43,44,45",https://github.com/uclouvain/openjpeg/commit/5...
867,https://github.com/uclouvain/openjpeg.git,5d00b719f4b93b1445e6fb4c766b9a9883c57949,static INLINE OPJ_BOOL opj_tcd_init_tile(opj_t...,1,False,"46,47,48,49",https://github.com/uclouvain/openjpeg/commit/5...
903,https://github.com/uclouvain/openjpeg.git,940100c28ae28931722290794889cf84a92c5f6f,static OPJ_BOOL opj_j2k_write_mco( opj_j2k...,1,False,"17,18,44,47",https://github.com/uclouvain/openjpeg/commit/9...


## Remove duplicate codes

In [ ]:
print("Before drop duplicates: ", df_codes.shape)

Before drop duplicates:  (200413, 7)


In [ ]:
# Sort by 'label' in descending order
df_codes = df_codes.sort_values(by='target', ascending=False)

df_codes.drop_duplicates(subset=['code_before'], keep='first', inplace=True)

In [ ]:
print("After drop duplicates: ", df_codes.shape)

After drop duplicates:  (134226, 7)


## Add tags of flaw lines to the code in order to recover in the later

In [ ]:
import re

FLAW_TAG_SYMBOL = '\u25C6'
POSITION = 'BEGIN'

def insert_tag(brow):
    row = brow.copy()
    # insert a rare token after the first no-space character to indicate the lines tagged by mod_lines in the code
    if row['mod_lines'] != '':
        mod_lines = row['mod_lines'].split(',')
        code = row['code_before']
        code = code.split('\n')
        for line in mod_lines:
            line = int(line)
            if POSITION == 'BEGIN':
                code[line] = re.sub(r"^(\s*\S)", fr"\1{FLAW_TAG_SYMBOL}", code[line])
            else:
                code[line] = code[line] + FLAW_TAG_SYMBOL

        code = '\n'.join(code)
        row['code_before'] = code
    return row

def remove_tag_and_generate_mod_lines(brow):
    row = brow.copy()
    # remove the rare token and generate mod_lines
    pattern = fr"^(\s*\S)\s*{FLAW_TAG_SYMBOL}"
    
    code = row['code_before']
    code = code.split('\n')
    mod_lines = []
    for i in range(len(code)):
        if bool(re.search(FLAW_TAG_SYMBOL, code[i])):
            if POSITION == "BEGIN":
                code[i] = re.sub(pattern, r"\1", code[i])
            else:
                code[i] = code[i].replace(FLAW_TAG_SYMBOL, "")
            mod_lines.append(str(i))

    row['code_before'] = '\n'.join(code)
    row['mod_lines'] = ','.join(mod_lines)

    return row


## Statistic the token length

In [ ]:
df_codes['token_length'] = df_codes['code_before'].apply(lambda x: len(tokenizer.encode(x)))
print("Token length:")
print(df_codes['token_length'].describe())

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors


Token length:
count    134226.000000
mean        568.429753
std        1637.260024
min           2.000000
25%         107.000000
50%         231.000000
75%         531.000000
max      146722.000000
Name: token_length, dtype: float64


## Remove comments in the code

In [ ]:
from tree_sitter import Language, Parser


class TSParse:
    r'''Parse C/C++ code using tree-sitter AST.
    Example:
    code = """
        #include "ap_config.h"
        using namespace ContosoData;\n
        int main() {
            int a = 1; // comment
            /* comment // comment /*
            int r += 4;
            here */ int c = 2;
            int d = 4;
            int b /* comment */ = 2;
            int e = 5;
        }
    """
    print(TSParse("./treesitter_c_cpp.so").noisy(code, "cpp", True))
    '''
    noisy_query = {}
    noisy_query["cpp"] = """
            (comment) @comment
            (preproc_include) @include
            (using_declaration) @using
        """
    noisy_query["c"] = "(comment) @comment (preproc_include) @include"
    noisy_query["python"] = """
        (comment) @comment
        (expression_statement 
            (string) @matched-string)
        """
    noisy_query["java"] = "(line_comment) @linecomment (block_comment) @blockcomment"
    noisy_query["default"] = "(comment) @comment"

    def __init__(self, lang):
        """Initialize Tree-Sitter parsers."""
        lang = lang.lower()
        assert lang in ["c", "cpp", "python", "java", "javascript", "typescript", "c#"], "Language must be C/C++/Python/Java/JavaScript/TypeScript/C#"
        if lang == "c#":
            lang = "c_sharp"
        
        self.lang = lang
        self.parser_lang = Language('/hpcfs/users/a1232991/Data/my-languages.so', lang)
        self.parser = Parser()
        self.parser.set_language(self.parser_lang)

    def query(self, code, query):
        """Run query on code."""
        # Get AST
        tree = self.parser.parse(bytes(code, "utf8"))

        # Return results
        results = self.parser_lang.query(query).captures(tree.root_node)
        # print(results)
        return results

    def clean(self, code):
        """Remove comments and includes from code.
        Args:
            code (str): Code as a string with newlines
            lang (str): Can be C or C++
        """

        # Find comments and includes
        noisy_query = self.noisy_query[self.lang] if self.lang in self.noisy_query.keys() else self.noisy_query["default"]
        results = self.query(code, noisy_query)

        # Remove comments and includes based on string slicing
        code_new = code.splitlines()
        for i in results:
            sline, schar = i[0].start_point
            eline, echar = i[0].end_point
            # print('I=', i)
            if sline == eline:
                code_new[sline] = code_new[sline][:schar] + code_new[sline][echar:]
            else:
                for line_index in range(sline, min(len(code_new), eline + 1)):
                    if line_index == sline:
                        code_new[sline] = code_new[sline][:schar]
                    elif line_index == eline:
                        code_new[eline] = code_new[eline][echar:]
                    else:
                        code_new[line_index] = ""

        # Filter out empty lines
        non_empty_lines = [line for line in code_new if line.strip() != '']

        # Rejoin the non-empty lines to form cleaned code
        code_new = '\n'.join(non_empty_lines)
        
        return code_new
        
        
c_code = """#include "ap_config.h"
using namespace ContosoData;

int main() { //comment
    int a = 1; // comment
    /* comment // comment /*
    int r += 4;
    here */ int c = 2;
    int d = 4;
    int b /* comment */ = 2;
    int e = 5;
}
"""

python_code = """
'''ApexCodeUtility
    afaf
    aprint("Hello, #world!")  # This is a comment
    '''
def my_function():
                '''ApexCodeUtility
                afaf
                aprint("Hello, #world!")  # This is a comment
                '''
                print("Hello, #world!")  # This is a comment
                '''ApexCodeUtility
                afaf
                aprint("Hello, #world!")  # This is a comment
                '''
                x = "hello"
                y = "auto_shrink_indentation
                afafaf"
                x = 42  # Another comment
                # This line is a comment
                # This line is a comment
                # This line is a comment
"""

ts_code = """
case ts.SyntaxKind.VariableStatement:
            case ts.SyntaxKind.TypeAliasDeclaration:
            case ts.SyntaxKind.FunctionDeclaration:
            case ts.SyntaxKind.ModuleDeclaration:
                stop = visitor(node);
        }
        // if (node.kind !== ts.SyntaxKind.SourceFile) {
        // 	if (getNodeText(sourceFile, node).indexOf('SymbolKind') >= 0) {
        // 		console.log('FOUND TEXT IN NODE: ' + ts.SyntaxKind[node.kind]);
        // 		console.log(getNodeText(sourceFile, node));
        // 	}
        // }
        if (stop) {
            return;
        }
        ts.forEachChild(node, visit);
    };
    visit(sourceFile);
"""

java_code = """
public class LowerTriangular    
{    
    public static void main(String[] args) {    
    int rows, cols;   
    /*
    afaf
    afaf
    */
    //Initialize matrix a    
        int a[][] = {       
                        {1, 2, 3},    
                        {8, 6, 4},    
                        {4, 5, 6}    
                    };    
              
          //Calculates number of rows and columns present in given matrix    
          rows = a.length;    
          cols = a[0].length;    
            
          if(rows != cols){    
              System.out.println("Matrix should be a square matrix");    
          }    
          else {    
              //Performs required operation to convert given matrix into lower triangular matrix    
              System.out.println("Lower triangular matrix: ");    
              for(int i = 0; i < rows; i++){    
                  for(int j = 0; j < cols; j++){    
                    if(j > i)    
                      System.out.print("0 ");    
                    else    
                      System.out.print(a[i][j] + " ");    
                }    
                System.out.println();    
            }    
        }    
    }    
}    """


parser = TSParse(language)
# parser = TSParse("python")
# clean_code = parser.clean(python_code)

# print(clean_code)


In [ ]:
if b_remove_comments:
    df_codes = df_codes.apply(insert_tag, axis=1)
    df_codes['code_before'] = df_codes['code_before'].apply(lambda x: parser.clean(x))
    df_codes = df_codes.apply(remove_tag_and_generate_mod_lines, axis=1)
    print("After removing comments")
    df_codes['token_length'] = df_codes['code_before'].apply(lambda x: len(tokenizer.encode(x)))
    print(df_codes['token_length'].describe())

After removing comments
count    134226.000000
mean        522.627293
std        1526.387956
min           2.000000
25%         103.000000
50%         218.000000
75%         486.000000
max      143832.000000
Name: token_length, dtype: float64


## Shrink the indentation

In [ ]:
# Not in use
import re

def code_format(row):
    name = f"{random.getrandbits(128)}.py"
    x = x.strip()
    with open(name, 'w') as f:
        f.write(x)
    
    # execute black through command line
    os.system(f"black {name}")

    # read back the formatted code
    with open(name, 'r') as f:
        x = f.read()
    os.remove(name)
    return x

def replace_whitespace(text):
    text = text.strip()
    return re.sub(r'\s+', ' ', text)

In [ ]:
import black
import textwrap
import re
import autopep8

def format_indentation(code):
    # Remove common leading whitespace
    code = textwrap.dedent(code)
    unit=' '

    try:
        code = black.format_str(code, mode=black.FileMode())
    except Exception as e:
        formatted_code = autopep8.fix_code(code)
        # try:
        #     code = convert_to_python3(code)
        #     code = black.format_str(code, mode=black.FileMode())
        # except:
        #     print("Error in conversion:", e, "code:", code)
        #     return None
    
    # Adjust the indentation level
    def adjust_indent(match):
        # For every 4 spaces, add a single unit
        return unit * (len(match.group(0)) // 4)

    # Replace every multiple of 4 spaces with the corresponding number of units
    formatted_code = re.sub(r'^( {4})+', adjust_indent, code, flags=re.MULTILINE)

    return formatted_code


# Test
python2_code = """
    def my_function():
            print "Hello World"
"""

# Test
code = """
def foo():
    if True:
        print "Hello" 
        if False:
            print("Nested")
    print("World")
"""

print(format_indentation(code))

# formatted_code = black.format_str(code, mode=black.FileMode())
# print(formatted_code)

# python3_code = convert_to_python3(python2_code)
# print(python3_code)


def foo():
 if True:
  print "Hello" 
  if False:
   print("Nested")
 print("World")



In [ ]:
import re
from math import gcd

def auto_shrink_indentation(code):
    code = code.strip()
    lines = code.split('\n')

    # Determine the indentation type (space or tab)
    space_indent_count = []
    tab_indent_count = []
    for line in lines:
        if line.startswith(' '):
            space_indent_count.append(len(re.match(r'^[ ]*', line).group()))
        elif line.startswith('\t'):
            tab_indent_count.append(len(re.match(r'^[\t]*', line).group()))

    indent_count = space_indent_count if len(space_indent_count) > 0 else tab_indent_count
    using_tabs = len(tab_indent_count) > 0
    # Calculate the greatest common divisor (GCD) of indentation quantities

    if len(indent_count) == 0:
        return code

    if language == "Python":
        # min_positive = min(x for x in indent_count if x > 0)
        # if min_positive == 2:
        #     indent_count = [x - x % 2 for x in indent_count]
        # else:
        #     indent_count = [x - x % 4 for x in indent_count]
        # gcd_indent = indent_count[0]

        # for num in indent_count[1:]:
        #     if num > 0:
        #         gcd_indent = gcd(gcd_indent, num)

        # if gcd_indent == 0:
        #     print(indent_count)
        #     print(code)
        # # print("GCD: ", gcd_indent)
        # # Shrink the indentation by dividing by the GCD
        # shrunk_lines = []
        # for line in lines:
        #     if line.startswith(' '):
        #         indent_count = len(re.match(r'^[ ]*', line).group())
        #         shrunk_indent_count = indent_count // gcd_indent
        #         shrunk_line = ' ' * shrunk_indent_count + line[indent_count:]
        #     elif line.startswith('\t'):
        #         indent_count = len(re.match(r'^[\t]*', line).group())
        #         shrunk_indent_count = indent_count // gcd_indent
        #         shrunk_line = '\t' * shrunk_indent_count + line[indent_count:]
        #     else:
        #         shrunk_line = line
        #     shrunk_lines.append(shrunk_line)

        # shrunk_code = '\n'.join(shrunk_lines)
        shrunk_code = format_indentation(code)
    else:
        if using_tabs:
            # If using tabs, replace consecutive tabs at the start of a line with a single tab
            shrunk_code = re.sub(r'^\t+', '\t', code, flags=re.MULTILINE)
        else:
            # If using spaces, replace consecutive spaces at the start of a line with a single space
            shrunk_code = re.sub(r'^ +', ' ', code, flags=re.MULTILINE)

    return shrunk_code

# Example usage:
original_code = """
def main():
 (opts, dummy) = parseArgs()

 if not os.path.exists(opts.destdir) and not opts.urls:
  try:
   os.makedirs(opts.destdir)
  except OSError, e:
   print >> sys.stderr, _("Error: Cannot create destination dir %s") % opts.destdir
   sys.exit(1)

 if not os.access(opts.destdir, os.W_OK) and not opts.urls:
  print >> sys.stderr, _("Error: Cannot write to  destination dir %s") % opts.destdir
  sys.exit(1)

 my = RepoSync(opts=opts)
 my.doConfigSetup(fn=opts.config, init_plugins=opts.plugins)

 # Force unprivileged users to have a private temporary cachedir
 # if they've not given an explicit cachedir
 if os.getuid() != 0 and not opts.cachedir:
  opts.tempcache = True

"""
# language = "Java"
# original_code = convert_to_python3(original_code)
# shrunken_code = auto_shrink_indentation(original_code)
# code = convert_to_python3(original_code)
# code = black.format_str(code, mode=black.FileMode())
# print(format_indentation(original_code))

# code_list = df_codes['code_before'].to_list()
# for code in code_list:
#     code = format_indentation(code)
#     if code == None:
#         break



In [ ]:
if b_shrink_indentation:
    if language == "Python":
        FLAW_TAG_SYMBOL = '## FLAW_LINE_TAG ##'
        POSITION = 'END'
        
    df_codes = df_codes.apply(insert_tag, axis=1)
    df_codes['code_before'] = df_codes['code_before'].apply(auto_shrink_indentation)
    df_codes = df_codes.apply(remove_tag_and_generate_mod_lines, axis=1)

    df_codes['token_length'] = df_codes['code_before'].apply(lambda x: len(tokenizer.encode(x)))
    print("After shrinking indentation")
    print(df_codes['token_length'].describe())
# df_codes['code_before'] = df_codes['code_before'].apply(lambda x: x.strip())

After shrinking indentation
count    134226.000000
mean        423.933724
std        1047.244801
min           2.000000
25%          96.000000
50%         196.000000
75%         423.000000
max       84663.000000
Name: token_length, dtype: float64


In [ ]:
# Sort the DataFrame in ascending order
df_codes.sort_values(by='token_length', inplace=True)

# Specify the desired percentage  RC&SI 93: 513 / None 82.4: 513
desired_percentage = 80

# Calculate the median index
median_index = int((desired_percentage / 100) * len(df_codes))

# Get the median value
median_value = df_codes['token_length'].iloc[median_index]

print(median_value)

514


## Whether remove the codes with token length > 512

In [ ]:
# df_codes = df_codes[(df_codes['added_only'] == "False") | (df_codes['token_length'] <= 512)]
# df_codes = df_codes[df_codes['token_length'] <= 512]

In [ ]:
df_codes[df_codes['added_only'] == "True"]['token_length'].describe()

count     3301.000000
mean      1030.276583
std       2749.440791
min         20.000000
25%        204.000000
50%        463.000000
75%       1004.000000
max      84663.000000
Name: token_length, dtype: float64

In [ ]:
df_codes.head()

,repo_url,hash,code_before,target,added_only,mod_lines,commit_url,token_length
154038,https://github.com/torvalds/linux.git,638164a2718f337ea224b747cf5977ef143166a4,,0,False,,https://github.com/torvalds/linux/commit/63816...,2
135611,https://github.com/torvalds/linux.git,82939d7999dfc1f1998c4b1c12e2f19edbdff272,},0,False,,https://github.com/torvalds/linux/commit/82939...,3
135607,https://github.com/torvalds/linux.git,82939d7999dfc1f1998c4b1c12e2f19edbdff272,#else,0,False,,https://github.com/torvalds/linux/commit/82939...,4
179928,https://github.com/lldpd/lldpd.git,9221b5c249f9e4843f77c7f888d5705348d179c0,static void version_check(void) {},0,False,,https://github.com/lldpd/lldpd/commit/9221b5c2...,11
55987,https://github.com/tats/w3m.git,18dcbadf2771cdb0c18509b14e4e73505b242753,pcmap(void)\n{\n},0,False,,https://github.com/tats/w3m/commit/18dcbadf277...,11


In [ ]:
print(df_codes.iloc[4]['code_before'])

pcmap(void)
{
}


In [ ]:
idx = 132
print(df_codes[df_codes['target']==1].iloc[idx]['code_before'])
print(df_codes[df_codes['target']==1].iloc[idx]['mod_lines'])
print(df_codes[df_codes['target']==1].iloc[idx]['added_only'])

static inline struct f2fs_sb_info *F2FS_P_SB(struct page *page)
{
	return F2FS_M_SB(page->mapping);
}
2
False


In [ ]:
df_codes.iloc[0]['repo_url'] + '/commit/'+df_codes.iloc[0]['hash']

'https://github.com/torvalds/linux.git/commit/638164a2718f337ea224b747cf5977ef143166a4'

## Any N/A lines?

In [ ]:
df_codes.isna().sum().sum()

0

In [ ]:
# if 'flaw_lines' in df_codes.columns:
#     df_flaw_index = df_codes.apply(
#         lambda row: ",".join([str(int(i) - int(row['start_line'])) for i in row['flaw_lines'].split(",")])
#                     if row['target'] == 1 else -1,
#         axis=1
#     )

#     # Update the flaw_index column in the DataFrame
#     df_codes['flaw_index'] = df_flaw_index
# else:
#     df_codes['flaw_index'] = -1
#     df_codes['flaw_lines'] = -1
df_codes['flaw_index'] = -1

In [ ]:
from sklearn.model_selection import train_test_split

os.makedirs("data", exist_ok=True)
df_python_align = df_codes[['code_before', 'target', 'mod_lines', 'flaw_index', 'commit_url']]
df_python_align.columns = ['processed_func', 'target', 'flaw_line', 'flaw_line_index', 'commit_url']

# Split the DataFrame into train and remaining
train_df, remaining_df = train_test_split(df_python_align, test_size=0.2, random_state=1)

# Split the remaining DataFrame into validation and test
val_df, test_df = train_test_split(remaining_df, test_size=0.5, random_state=1)

# Print the shapes of the resulting DataFrames
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

train_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE_{sname}/CVE{language}_train2.csv", index=None)
val_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE_{sname}/CVE{language}_val2.csv", index=None)
test_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE_{sname}/CVE{language}_test2.csv", index=None)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Train set shape: (107380, 5)
Validation set shape: (13423, 5)
Test set shape: (13423, 5)


In [ ]:
python_code = """
writeToMethod(SWFBlock block, 
                           SWFByteOutputMethod method, void* data)
{
	SWFOutput out = ((SWFShape)block)->out;
	SWFOutput_writeToMethod(out, method, data);
}
"""

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
results1 = tokenizer.encode(python_code)
# print in one line
print(results1)

[0, 50118, 29631, 3972, 47967, 1640, 11871, 597, 38866, 1803, 6, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 8883, 597, 47447, 48293, 47967, 5448, 6, 13842, 3226, 414, 43, 50118, 45152, 50118, 50117, 11871, 7942, 1182, 9179, 66, 5457, 41006, 11871, 597, 45336, 43, 16776, 43, 46613, 995, 131, 50118, 50117, 11871, 7942, 1182, 9179, 1215, 29631, 3972, 47967, 1640, 995, 6, 5448, 6, 414, 4397, 50118, 24303, 50118, 2]


In [ ]:
len(results1)

91